In [ ]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# STEP 2: Set paths for sugarcane
dataset_path = '/content/drive/MyDrive/PlantDisease/sugarcane'  # Update for sugarcane dataset
model_save_path = '/content/drive/MyDrive/savedModels/sugarcane_disease_model.h5'

# STEP 3: Import libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# STEP 4: Prepare data generators
img_size = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

# STEP 5: Load MobileNetV2 base
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False  # Freeze base

# STEP 6: Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# STEP 7: Compile model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# STEP 8: Train model
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)

# STEP 9: Save trained model
model.save(model_save_path)
print("✅ Sugarcane disease model saved at:", model_save_path)


Mounted at /content/drive
Found 2023 images belonging to 5 classes.
Found 503 images belonging to 5 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 1323s 21s/step - accuracy: 0.2163 - loss: 1.9556 - val_accuracy: 0.2505 - val_loss: 1.6976
Epoch 2/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 164s 3s/step - accuracy: 0.3279 - loss: 1.6508 - val_accuracy: 0.3121 - val_loss: 1.5744
Epoch 3/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 158s 2s/step - accuracy: 0.3798 - loss: 1.5095 - val_accuracy: 0.4115 - val_loss: 1.4268
Epoch 4/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 165s 3s/step - accuracy: 0.4648 - loss: 1.3155 - val_accuracy: 0.4314 - val_loss: 1.3512
Epoch 5/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 164s 3s/step - accuracy: 0.4912 - loss: 1.2641 - val_accuracy: 0.4652 - val_loss: 1.2931
Epoch 6/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 163s 3s/step - accuracy: 0.5385 - loss: 1.1688 - val_accuracy: 0.4751 - val_loss: 1.2346
Epoch 7/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 163s 3s/step - accuracy: 0.5764 - loss: 1.0702 - val_accuracy: 0.5030 - val_loss: 1.2020
Epoch 8/15
64/64 ━━━━━━━━━━━━━━━━━━━━ 170s 3s/step - accuracy: 0.6110 - loss: 1.0211 - val_accuracy: 0.5487 -

✅ Sugarcane disease model saved at: /content/drive/MyDrive/savedModels/sugarcane_disease_model.h5


In [ ]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# STEP 2: Upload a custom image for prediction
from google.colab import files
uploaded = files.upload()

import os
image_path = list(uploaded.keys())[0]  # Name of the uploaded image

# STEP 3: Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# STEP 4: Load the trained sugarcane model
model = load_model('/content/drive/MyDrive/savedModels/sugarcane_disease_model.h5')  # ✅ Adjust path if needed

# STEP 5: Preprocess the uploaded image
img = image.load_img(image_path, target_size=(224, 224))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# STEP 6: Predict the disease class
pred = model.predict(img_array)
class_index = np.argmax(pred[0])
confidence = np.max(pred[0]) * 100

# STEP 7: Class labels and fertilizer suggestions (based on your folder structure)
class_labels = ['Healthy', 'Mosaic', 'RedRot', 'Rust', 'Yellow']  # ✅ Matches the folder names in your dataset

fertilizer_suggestions = {
    'Healthy': '✅ No disease detected. Maintain regular fertilization with NPK 18-18-18.',
    'Mosaic': '🦠 Control aphid vectors using insecticides. Use virus-free planting materials.',
    'RedRot': '🛡 Apply fungicides like Carbendazim. Remove infected stalks and practice crop rotation.',
    'Rust': '🌫 Use fungicides like Mancozeb. Ensure good air circulation and proper drainage.',
    'Yellow': '🌿 Apply micronutrients especially sulfur and zinc. Check for nutrient deficiency and pests.'
}

# STEP 8: Output the prediction and fertilizer suggestion
predicted_label = class_labels[class_index]
suggestion = fertilizer_suggestions.get(predicted_label, "No specific suggestion available.")

print(f"🍬 Predicted Disease: {predicted_label}")
print(f"✅ Confidence: {confidence:.2f}%")
print(f"💡 Fertilizer Suggestion: {suggestion}")

# STEP 9: Display the uploaded image
plt.imshow(img)
plt.axis('off')
plt.title(f"{predicted_label} ({confidence:.2f}%)")
plt.show()
